CNN에서는 "__패딩(Padding)__", "__스트라이드(Stride)__"등 CNN 고유의 용어가 등장한다. 또, 각 계층 사이에는 3차원 데이터같이 입체적인 데이터가 흐른다는 점에서 완전연결 신경망과 다르다.

# 7.2.1 완전연결 계층의 문제점

지금까지 본 오나전연결 신경망에서는 완전연결 계층(Affine 계층)을 사용했다. 완전연결 계층에서는 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있었다.

완전연결 계층의 문제점은 "데이터의 형상이 무시"된다는 사실이다. 입력 데이터가 이미지인 경우를 예로 들면, 이미지는 통상 세로, 가로, 채널(색상)로 구성된 3차원 데이터이다. 그러나 완전연결 계층에 입력할 때는 3차원 데이터를 평평한 1차원 데이터로 평탄화해줘야한다. 사실 지금까지 MNIST데이터셋을 이용한 사례에서는 형상이 (1,28,28)인 이미지(1채널, 세로 28픽셀, 가로 28픽셀)를 1줄로 세운 784개의 데이터를 첫 Affine 계층에 입력했다.

이미지는 3차원 형상이며, 이 형상에는 소중한 공간적 정보가 담겨 있다. 예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접하게 연관되어 있거나, 거리가 먼 픽셀끼리는 별 연관이 없는 등, 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있을 것이다. 그러나 완전연결 계층은 형상을 무시하고 모든 입력 데이터를 동등한 뉴런(같은 차원의 뉴런)으로 취급하여 형상에 담긴 정보를 살릴 수 없다.

한편, 합성곱 계층은 형상을 유지한다. 이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달한다. 그래서 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할 것이다.

CNN에서는 합성곱 계층의 입출력 데이터를 "__특징 맵(feature map)__"이라고 한다. 합성곱 계층의 입력 데이터를 "__입력 특징 맵(input feature map)__", 출력 데이터를 "__출력 특징 맵(output feature map)__"이라고 하는 식이다. 이 자료에서는 "입출력 데이터"와 "특징 맵"을 같은 의미로 사용한다.

# 7.2.2 합성곱 연산

합성곱 계층에서의 "__합성곱 연산__"을 처리한다. 합성곱 연산은 이미지 처리에서 말하는 "__필터 연산__"에 해당한다. 구체적인 예를 들어 보자.

<img src=images/7_03.png height=100px width=500px>

위와 같이 합성곱 연산은 입력 데이터에 필터를 적용한다. 이 예에서 입력 데이터는 세로, 가로 방향의 형상을 가졌고, 필터 역시 세로, 가로 방향의 차원을 갖는다. 데이터와 필터의 형상을 $(높이^{weight}, 너비^{width})$로 표기하며, 이 예에서는 입력은 (4,4), 필터는 (3,3), 출력은 (2,2)가 된다. 문헌에 따라 필터를 "__커널__"이라고 칭하기도 한다.

그럼 위 그림의 합성곱 연산 예에서 어떤 계산이 이루어지는지 확인해보자. 다음 그림은 합성곱 연산의 계산 순서를 그려본 것이다. 합성곱 연산은 필터의 "__윈도우(window)__"를 일정 간격으로 이동해가며 입력 데이터에 적용한다. 여기에서 말하는 윈도우는 그림의 회색 (3,3) 부분을 가리킨다. 이 그림에서 보듯 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구한다(이 계산을 "__단일 곱셈-누산(fused multiply-add, FMA)__"라 한다). 그리고 그 결과를 출력의 해당 장소에 저장한다. 이 과정을 모든 장소에서 수행하면 합성곱 연산의 출력이 완성된다.

<img src=images/7_04.png height=100px width=500px>

완전연결 신경망에는 가중치 매개변수와 편향이 존재하는데, CNN에서는 필터의 매개변수가 그 동안의 "가중치"에 해당한다. 그리고 CNN에도 편향이 존재한다. 위는 필터를 적용하는 단계까지만 보여준 것이고, 편향까지 포함하면 다음과 같은 흐름이 된다.

<img src=images/7_05.png height=100px width=500px>

이와 같이 편향은 필터를 적용한 후의 데이터에 더해진다. 그리고 편향은 항상 하나(1,1)만 존재한다. 그 하나의 값을 필터를 적용한 모든 원소에 더하는 것이다.

# 7.2.3 패딩

합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값 (예컨대 0)으로 채우기도 한다. 이를 "__패딩(padding)__"이라 하며, 합성곱 연산에서 자주 이용하는 기법이다. 예를 들어 다음은 (4,4) 크기의 입력 데이터에 폭이 1인 패딩을 적용한 모습이다. 폭 1짜리 패딩이라 하면 입력 데이터 사방 1픽셀을 특정 값으로 채우는 것이다.

<img src=images/7_06.png height=100px width=500px>

이와 같이 처음에 크기가 (4,4)인 입력 데이터에 패딩이 추가되어 (6,6)이 된다. 이 입력에 (3,3) 크기의 필터를 걸면 (4,4) 크기의 출력 데이터가 생성된다. 이 예에서는 패딩을 1로 설정했지만, 2나 3 등 원하는 정수로 설정할 수 있다. 만약 위 편향 그림에 패딩을 2로 설정하면 입력 데이터의 크기는 (8,8)이 되고 3으로 설정하면 (10,10)이 된다.

패딩은 주로 출력 크기를 조정할 목적으로 사용된다. 예를 들어 (4,4) 입력 데이터에 (3,3) 필터를 적용하면 출력은 (2,2)가 되어, 입력보다 2만큼 줄어든다. 이는 합성곱 연산을 몇번이나 되풀이하는 심층 신경망에서는 문제가 될 수 있다. 이러한 사태를 막기 위해 패딩을 사용한다. 바로 위 예에서는 (4,4) 입력 데이터에 패딩을 1만큼 적용하니 출력 데이터가 (3,3)필터에 대해 (4,4)로 유지되는 모습을 볼 수 있다.

# 7.2.4 스트라이드

필터를 적용하는 위치의 간격을 "__스트라이드(stride)__"라고 한다. 지금까지 본 예는 모두 스트라이드가 1이 었지만, 예를 들어 스트라이드를 2로 하면 필터를 적용하는 윈도우가 두 칸씩 이동한다.

<img src=images/7_07.png height=100px width=500px>

이 그림에서는 크기가 (7,7)인 입력 데이터에 스트라이드를 2로 설정한 필터를 적용한다. 이처럼 스트라이드는 필터를 적용하는 간격을 지정한다.

그런데, 스트라이드를 2로 하니 출력은 (3,3)이 된다. 이처럼 스트라이드를 키우면 출력 크기는 작아진다. 한편, 패딩을 크게하면 출력 크기가 커졌다. 이러한 관계를 수식화 하면 다음과 같다. 

입력 크기를 (H, W), 필터 크기를 (FH, FW), 출력 크기를 (OH, OW), 패딩을 P, 스트라이드를 S라 하면, 출력 크기는 다음 식으로 계산된다.

$
OH = \frac{H+2P-FH}{S} + 1 \\
OW = \frac{W+2P-FW}{S} + 1 \\
\qquad$ [식 7.1]

[식 7.1]에 단순히 값을 대입하기만 하면 출력 크기를 구할 수 있다. 단, OH와 OW는 정수로 나눠떨어지는 값이어야 한다. 출력 크기가 정수가 아니면 오류를 내는 등의 대응을 해줘야할 것이다. 덧붙여서 딥러닝 프레임워크 중에는 값이 딱 나눠떨어지지 않을 때는 가장 가까운 정수로 반올림하는 등, 특별히 에러를 내지않고 진행하도록 구현하는 경우도 있다.
CNN에 대한 더 자세한 설명은 이 링크에 담겨있다. 

[CS231n Convolutional Neural Networks for Visual Recognition](https://cs231n.github.io/convolutional-networks/)

# 7.2.5 3차원 데이터의 합성곱 연산

지금까지 2차원 형상을 다루는 합성곱 연산을 살펴보았다. 그러나 이미지만 해도 세로, 가로에 더해서 채널까지 고려한 3차원 데이터이다. 이번 절에서는 조금 전과 같은 순서로, 채널까지 고려한 3차원 데이터를 다루는 합성곱 연산을 살펴보자. 

다음은 3차원 데이터의 합성곱 연산 예이다. 그리고 그 다음 그림은 계산 순서이다. 2차원일 때와 비교하면, 길이 방향으로 특징 맵이 늘어났다. 채널쪽으로 특징 맵이 여러 개 있다면 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 하나의 출력을 얻는다.

<img src=images/7_08.png height=100px width=500px>

___

<img src=images/7_09.png height=100px width=500px>

3차원의 합성곱 연산에서 주의할 점은 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 것이다. 이 예에서는 모두 3개로 같다. 판현, 필터 자체의 크기는 원하는 값으로 설정할 수 있다(단, 모든 채널의 필터가 같은 크기여야 한다). 

# 7.2.6 블록으로 생각하기

3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉽다. 블록은 다음과 같은 3차원 직육면체이다. 또, 3차원 데이터를 다차원 배열로 나타낼 때는 $(채널^{channel}, 높이^{height}, 너비^{width})$순서로 쓰겠다. 

<img src=images/7_10.png height=100px width=500px>

이 예에서 출력 데이터는 한 장의 특징 맵이다. 한 장의 특징 맵을 다른 말로 하면 채널이 1개인 특징 맵이다. 그럼 합성곱 연산의 출력으로 다수의 채널을 내보내려면 어떻게 해야 할까? 그 답은 필터(가중치)를 다수 사용하는 것이다. 그림으로는 다음과 같다.

<img src=images/7_11.png height=100px width=500px>

이 그림과 같이 필터를 FN개 적용하면 출력 맵도 FN개 가 생성된다. 그리고 그 FN개의 맵을 모으면 형상이 (FN, OH, OW)인 블록이 완성된다. 이 완성된 블록을 다음 계층으로 넘기겠다는 것이 CNN의 흐름이다. 이상에서 보듯 합성곱 연산에서는 필터의 수도 고려해야 한다. 그런 이유로 필터의 가중치 데이터는 4차원 데이터이며(출력 채널 수, 입력 채널 수, 높이, 너비)순으로 쓴다. 

합성곱 연산에도 편향이 쓰인다. 다음 그림은 위 그림에 편향을 더한 모습이다.

<img src=images/7_12.png height=100px width=500px>

위에서 보듯 편향은 채널 하나에 값 하나씩으로 구성된다. 이 예에서는 편향의 형상은 (FN, 1, 1)이고, 필터의 출력 결과의 형상은 (FN, OH, OW)이다. 이 두 블록을 더하면 편향의 각 값이 필터의 출력인 (FN, OH, OW) 블록의 대응 채널의 원소 모두에 더해진다. 참고로, 형상이 다른 블록의 덧셈은 넘파이의 브로드캐스트 기능으로 쉽게 구현할 수 있다.

# 7.2.7 배치 처리

신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리했다. 완전연결 신경망을 구현하면서는 이 방식을 지원하여 처리 효율을 높이고, 미니배치 방식의 학습도 지원하도록 했다. 

합성곱 연산도 마찬가지로 배치 처리를 지원하고자 한다. 그래서 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장한다. 구체적으로는 데이터를 (데이터 수, 채널 수, 높이, 너비) 순으로 저장한다. 데이터가 N개일 때 위의 편향을 포함한 연산을 배치 처리한다면 데이터의 형태는 다음과 같다.

<img src=images/7_13.png height=100px width=500px>

배치 처리 시의 데이터 흐름을 나타낸 위 그림을 보면, 각 데이터의 선두에 배치용 차원을 추가하였다. 이처럼 데이터는 4차원 형상을 가진 채 각 계층을 타고 흐른다. 여기에서 주의할 점으로는 신경망에 4차원 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다는 것이다. 즉, N회 분의 처리를 한 번에 수행하는 것이다.